<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC581B - Introduction to Deep Learning for Image Classification
# Final Project

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

First we need to load the training and test data.

In [2]:
# Load the training data (CIFAR10 to start)
training_data = datasets.CIFAR10(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# Load the test data
test_data = datasets.CIFAR10(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

'''training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)'''

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


'training_data = datasets.FashionMNIST(\n    root = "data",\n    train = True,\n    download = True,\n    transform = ToTensor()\n)\n\ntest_data = datasets.FashionMNIST(\n    root = "data",\n    train = False,\n    download = True,\n    transform = ToTensor()\n)'

In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 3, 32, 32])
Shape of y:  torch.Size([64]) torch.int64


## Building a first model

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
# Define the model
class LinearModel(nn.Module):
  def __init__(self, n_neurons):
    super(LinearModel, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_model = nn.Sequential(
        nn.Linear(32*32*3, n_neurons),
        nn.Linear(n_neurons, n_neurons),
        nn.Linear(n_neurons, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_model(x)
    return logits

In [10]:
model = LinearModel(n_neurons=4096).to(device)
print(model)

LinearModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_model): Sequential(
    (0): Linear(in_features=3072, out_features=4096, bias=True)
    (1): Linear(in_features=4096, out_features=4096, bias=True)
    (2): Linear(in_features=4096, out_features=10, bias=True)
  )
)


In [11]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
# Define training loop function
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [13]:
# Define the test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
n_epochs = 5

In [15]:
for t in range(n_epochs):
  print(f"Epoch {t+1}\n----------------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done.")

Epoch 1
----------------------------------------
loss: 2.311939 [    0/50000]
loss: 2.285006 [ 6400/50000]
loss: 2.213299 [12800/50000]
loss: 2.239101 [19200/50000]
loss: 2.182528 [25600/50000]
loss: 2.172086 [32000/50000]
loss: 2.211947 [38400/50000]
loss: 2.128653 [44800/50000]
Test Error: 
 Accuracy: 25.9%, Avg loss: 2.124596 

Epoch 2
----------------------------------------
loss: 2.172225 [    0/50000]
loss: 2.140723 [ 6400/50000]
loss: 1.999905 [12800/50000]
loss: 2.139411 [19200/50000]
loss: 2.052738 [25600/50000]
loss: 2.052850 [32000/50000]
loss: 2.159014 [38400/50000]
loss: 2.021031 [44800/50000]
Test Error: 
 Accuracy: 29.5%, Avg loss: 2.030582 

Epoch 3
----------------------------------------
loss: 2.089527 [    0/50000]
loss: 2.061327 [ 6400/50000]
loss: 1.875973 [12800/50000]
loss: 2.073730 [19200/50000]
loss: 1.999727 [25600/50000]
loss: 1.989661 [32000/50000]
loss: 2.124464 [38400/50000]
loss: 1.961567 [44800/50000]
Test Error: 
 Accuracy: 30.9%, Avg loss: 1.974369 

E

In [ ]:
# Save the model
torch.save(model.state_dict(), "linear_model.pth")